# Workshop №2

## Черновик того, что должно оказаться затем в Airflow

### Содержание DAGа

1. [Первая таска по забору данных от API](#first)
2. [Вторая таска - сенсор по готовности даннных](#second)
3. [Третья таска - brunch оператор по выбору типа загрузки](#third)
4. [Четвертая таска - загрузка обработанных данных](#fourth)

In [ ]:
import logging as log
import requests
import boto3
import json
import pandas as pd
from typing import Optional
from pyarrow import fs

---
### Наш первая таска: PythonOperator по загрузке данных в S3<a id="first"></a>
---

#### Функция получения данныx от API

https://exchangerate.host/#/docs - ссылка на апишку

Посмотрим, какие данные нам отдает API за текущий день

In [ ]:
response = requests.get('https://api.exchangerate.host/latest',
                        params={'base': 'USD', 'symbols': 'RUB'})
data = response.json()
data

За интервал времени

In [ ]:
response = requests.get('https://api.exchangerate.host/timeseries?start_date=2022-03-01&end_date=2022-03-10',
                        params={'base': 'USD', 'symbols': 'RUB'})
data = response.json()
data

Получается функция, которая может забирать данные за сегодняший день и может забирать историчные данные

In [ ]:
def get_currency_rate_info(source_currency: str, target_currency: str, is_initial: bool = False,
                           start_date: Optional[str] = None,
                           end_date: Optional[str] = None) -> dict:
    """
    Get currency rate information about pair

    :param source_currency: source currency
    :param target_currency: target currency
    :param is_initial: upload historical data. Default: False
    :param start_date: start date for uploading data. Default: None
    :param end_date: end date for uploading data. Default: None
    
    :return data: information about rate of pair
    """
    params: dict = {'base': source_currency, 'symbols': target_currency}

    if is_initial:
        url: str = f'https://api.exchangerate.host/timeseries?start_date={start_date}&end_date={end_date}'
    else:
        url: str = 'https://api.exchangerate.host/latest'

    try:
        response = requests.get(url, params=params)
        data = response.json()
    except Exception as ex:
        print(f"Unable get currency rate info. Error: {ex}")
        raise ex

    return data

#### Функция создания клиента для подключения к S3

In [ ]:
def init_s3_client(s3_host: str, s3_access_key: str, s3_secret_key: str) -> boto3.client:
    """
    Initialization S3 client

    :param s3_host: host
    :param s3_access_key: access key
    :param s3_secret_key: secret key
    
    :return client: S3 client
    """
    client = boto3.client(
            's3',
            region_name='us-east-1',
            use_ssl=True,
            endpoint_url=s3_host,
            aws_access_key_id=s3_access_key,
            aws_secret_access_key=s3_secret_key)
    return client

Попробуем получить данные и положить их в S3

In [ ]:
raw_data = get_currency_rate_info('USD','RUB')
s3_client = init_s3_client('http://127.0.0.1:9000', 'minioadmin', 'minioadmin')
converted_raw_data = json.dumps(raw_data) #сериализуем наш словарь в жисон строку

resp = s3_client.put_object(Bucket='practicum', Key='data/exchange_rate/raw/usd_rub/file.json', Body=converted_raw_data)
status_code = resp.get('ResponseMetadata').get('HTTPStatusCode')

if status_code == 200:
    print('File uploaded successfully')
else:
    print('File not uploaded')


На этом моменте наш оператор заканчивается

---
### Наша вторая таска: S3 sensor - проверяет данные в S3<a id="second"></a>
---

Представим, что не мы складываем себе raw данные, а складывает к нам в Data Lake соседняя команда, поэтому нам понадобится сенсор, чтобы проверять есть ли данные или нет для дальнейшей обработки

Можно воспользоваться готовым решением в [Airflow](https://airflow.apache.org/docs/apache-airflow-providers-amazon/stable/_api/airflow/providers/amazon/aws/sensors/s3/index.html#module-airflow.providers.amazon.aws.sensors.s3), сейчас есть на любой вкус операторы, сенсоры и прочее. Вот, например, для S3 - 

Но мы напишем свой, потому что мы можем:)

*Логика будет следующая:*
Создадим клиент для S3 и залистим все объекты по нашему префиксу

In [ ]:
client = init_s3_client('http://127.0.0.1:9000', 'minioadmin', 'minioadmin')
bucket_objects = client.list_objects(Bucket='practicum', Prefix='data/exchange_rate/raw/usd_rub/file.json')
bucket_objects

Надо выбрать условие, по которому будет понятно, есть ли данные или нет, для этого на поможет поле **Contents**

In [ ]:
bucket_objects.get('Contents')

#### Будущий кусок функции для проверки данных в S3

In [ ]:
client = init_s3_client('http://127.0.0.1:9000', 'minioadmin', 'minioadmin')

bucket_objects = client.list_objects(Bucket='practicum', Prefix='data/exchange_rate/raw/usd_rub/file.json')
if bucket_objects.get('Contents'):
    print(f'Data were found')
else:
    print(f'Data not found')

---
### Наша третья таска: BrunchPythonOperator - она просто выбирает между тасками<a id="third"></a>
---

Чтобы продемонстрировать вам brunch operator представим, что нам нужно выбирать между архивной прогрузкой или нет, данный оператор будет нам помогать в этом. Если наш любимый флаг is_initial - True, значит будет архивная таска запускаться, если False - значит ежедневная прогрузка будет

---
### Наша четвертая и пятая таска: PythonOperator - initial и no initial<a id="fourth"></a>
---

Получим данные из raw слоя:

In [ ]:
s3_client = init_s3_client('http://127.0.0.1:9000', 'minioadmin', 'minioadmin')
obj = s3_client.get_object(Bucket='practicum', Key='data/exchange_rate/raw/usd_rub/file.json')
raw_object_from_s3 = json.loads(obj['Body'].read())
raw_object_from_s3

Нашим DSам нужна только дата и курс, остальные поля не нужны

In [ ]:
is_initial = False
parsed_data = []
if not is_initial:
    parsed_data.append([raw_object_from_s3['date'], raw_object_from_s3['rates']['RUB']])
else:
    for key, value in raw_object_from_s3['rates'].items():
        parsed_data.append([key, value['RUB']])
parsed_data

Теперь нам это нужно прогрузить в слой parsed, но на этот раз, не в жисонах будем складывать, а в паркетах

In [ ]:
s3_fs = fs.S3FileSystem(endpoint_override='http://127.0.0.1:9000', access_key='minioadmin', secret_key='minioadmin')
df = pd.DataFrame(parsed_data, columns = ['date', 'rate'])
df.to_parquet("practicum/data/parsed/my_file.parquet", engine='pyarrow', filesystem=s3_fs)